# PySPH tutorial: Getting started

*Prabhu Ramachandran*

Department of Aerospace Engineering, IIT Bombay

----


## Installation and getting started

This is a simple introduction to PySPH.  The PySPH documentation is here: http://pysph.readthedocs.io

1. First install PySPH.  See here: http://pysph.readthedocs.io/en/latest/installation.html
2. Go over the installation and getting started page


Importantly, once you have PySPH installed run a simple example, as so:

    $ pysph run elliptical_drop
    
or

    $ pysph run rigid_body.bouncing_cubes
    
    
Then view the generated output:

    $ pysph view elliptical_drop_output
    
    
If this produces a nice looking view, you should be mostly set.  It may be handy to be able to run pysph on openmp:

    $ pysph run elliptical_drop --openmp

If you get this far and everything works, you should be in good shape.


## More on the examples


The examples are all written in pure Python.  To see the sources for the examples you could either visit the github sources here: https://github.com/pypr/pysph/tree/master/pysph/examples

Alternatively try this:

    $ pysph run
    
Now you can pick among 40 odd examples.  To see the source of a simple one you can do the following:


    $ pysph run --cat elliptical_drop
    
    
This will simply show you the source code without executing it.  You could have also run the example by changing directory into the `<pysph_root>/pysph/examples` directory and running the example, for example let us do this easily as follows:


    $ pysph run --cat elliptical_drop > ed.py    # This puts the source into ed.py in the current dir.
    
    $ python ed.py
    
**NOTE: **  there is also a `<pysph_root>/old_examples` directory which you should not use.
    
You can also import the examples from Python and thus could just as well have run this example as:

    $ python -m pysph.examples.elliptical_drop


The point I am making is that `pysph run` is not doing anything special at all, it just makes it a tad easier to run the examples.  These examples are usually quite useful and can also be subclassed if you wish to reuse them.




### Using the PySPH library


In order to simulate your own problems you need to understand the anatomy of a PySPH simulation.  All the examples typically will do the following:

1. Create some particles
2. Specify the equations for the inter-particle interactions.
3. Specify the integrators to use.
4. Put these together in an `Application` and run this application.


### Creating particles


In this tutorial we will mostly explore the creation of particles in PySPH.  In PySPH, particles are created in a data structure called a `ParticleArray`.  Let us consider an example.  Let us say we have a glass of water.  Clearly we have two "things", a glass vessel and the water inside it.  Since we wish to capture the interaction of the water with the vessel, we would create two `ParticleArray`s.  One for the vessel which we call `"solid"` and the other for the water which we call `"fluid"`.  

Some important points to note.  Each particle array 

- has a name (a string) which should be a valid Python variable name, `"fluid"` and `"solid"` are good as would be `"fluid1"` and `"fluid2"`.

- has a collection of various particle properties, like the position `x, y, z`, velocity components `u, v, w`, other scalar properties `m, h, rho` etc.  All of these properties are scalars.

- has a collection of "constants", which can have an arbitrary size but are internally stored as 1D arrays.

The properties are used for things that typically vary, from particle to particle.

The constants are used for things that are constant for all the particles.

Let us now try to create a particle array in order to understand it better.


In [ ]:
from __future__ import print_function

In [ ]:
from pysph.base.particle_array import ParticleArray

In [ ]:
pa = ParticleArray(name='fluid', x=[0.0, 1.0], y=0, m=0.1)
print(pa.name, pa.x, pa.y, pa.m)

Note that we set the name by a kwarg. 

#### Exercise

- Try creating a particle array without the name.
- While x was passed as a list, y and m were not, what is going on?
- Does this work with numpy arrays?  Try it!
- Does it work with numpy arrays of arbitrary shape?
- What if you have arrays passed of different sizes?!
- Can you add a new "crazy" property?


#### Solution

- You can create a particle array without a name but DON'T.
- NumPy arrays work and are ravelled, lists and constants work.
- Passing incompatible sizes is a problem and you will get an error.
- You can add any kind of property by passing a suitable kwarg.

In [ ]:
pa = ParticleArray(name='fluid', x=[0.0, 1.0], y=0, m=0.1, crazy=25)

In [ ]:
pa.crazy

### Doing more

- How do we discover the properties?
- Use `pa.properties`
- What about constants?


In [ ]:
pa.properties

So what are the other strange properties?  We didn't add `gid, pid and tag`

So it looks like PySPH automatically adds some special props, what are these?  

- gid: is a global ID for each particle, it is useful in parallel.
- pid: represents the process ID for each particle, also relevant in parallel.
- tag: represents the kind of particle we have.

The `tag` property is probably the most useful.  It is representative of the kind of particles, the ones that are important are:

- Local: see `get_local_tag` below
- Remote: see `get_remote_tag` below
- Ghost: see `get_ghost_tag` below


**Questions**

- What is this pyzoltan stuff??  
- What is a DoubleArray, IntArray?  



In [ ]:
pa.gid

In [ ]:
pa.pid

In [ ]:
pa.tag

In [ ]:
from pysph.base.particle_array import get_local_tag, get_ghost_tag, get_remote_tag

In [ ]:
print("Local:", get_local_tag())
print("Remote:", get_remote_tag())
print("Ghost:", get_ghost_tag())

#### Digression CArrays

Let us answer the question "What is this pyzoltan, DoubleArray stuff?"

These are internal arrays that allow us to efficiently store and compute with these properties and have some useful features.

In [ ]:
from pyzoltan.core.carray import DoubleArray

In [ ]:
a = DoubleArray(5)
x = a.get_npy_array()
x[:] = 100

In [ ]:
a[1]

In [ ]:
a.append(203)
a.length

#### Exercise

- Find the default properties.
- Can you create a particle array with no properties in the constructor?

In [ ]:
empty = ParticleArray(name='dummy')

In [ ]:
empty.properties

### Adding constants

Add them by passing a dictionary.

In [ ]:
pa = ParticleArray(name='fluid', x=[0.0, 1.0], constants={'rho0': 1000})
pa.rho0

In [ ]:
pa.constants

#### Exercises

- Create different kinds of constants and experiment
- Create a vector or a 2d array.  What happens to a 2d array?

In [ ]:
import numpy as np

In [ ]:
pa = ParticleArray(name='f', x=[0.0, 1.0], 
                   constants=
                   {'a': 1, 'b': [1,2,3], 'c': np.identity(10)})

In [ ]:
pa.constants

In [ ]:
pa.c

### Particle array methods

There are many methods but some more useful than others.  Let us explore this

- `pa.get_number_of_particles()`


- `pa.add_constant()`
- `pa.add_property(...)`
- `pa.add_particles()`

- `pa.extend(n)`
- `pa.extract_particles(...)`
- `pa.remove_particles()`
- `pa.remove_property(prop)`
- `pa.remove_tagged_particles(tag)`


The output property arrays is an important attribute.  It is what determines what is dumped to disk when you save particle arrays or run simulations.

- `pa.set_output_arrays(list)`
- `pa.output_property_arrays`
- `pa.add_output_arrays(list)`

####  Exercise

- Explore all of the above methods.


In [ ]:
pa.output_property_arrays

In [ ]:
pa.add_property('x')
pa.x = np.arange(10)

In [ ]:
pa.add_output_arrays(['x'])
pa.output_property_arrays

- `pa.get_carray(prop)`: will get you the c array.
- `pa.get(props)`: returns properties.
- `pa.set(**props)`: sets the properties in one go

### The convenient `pysph.base.utils`

- For many standard problems, one requires a bunch of additional properties.
- Use the `pysph.base.utils` for these.

In [ ]:
from pysph.base.utils import get_particle_array

In [ ]:
from pysph.base.utils import get_particle_array_wcsph, get_particle_array_tvf_fluid, get_particle_array_gasd

In [ ]:
pa = get_particle_array_wcsph(name='fluid', x=[1, 2], m=3)
pa.properties

#### Exercises

- Create particles inside a disk of radius 1.
- Visualize the particle positions.
- Create a WCSPH compatible particle array with these points.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

#### Solution

In [ ]:
%load solutions/particles_in_disk.py